In [343]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import sys, os, math, sqlite3
import emosent, emoji, string
from nltk import sentiment
import datetime, time, re, string

In [379]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Kyle\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [377]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag

In [98]:
expansive_emoji_dict = emoji.UNICODE_EMOJI

In [99]:
available_emoji_sentiments = emosent.EMOJI_SENTIMENT_DICT

In [108]:
sent_dict = {"Symbol":[], "Score":[], "Name":[]}
for k, v in available_emoji_sentiments.items():
    sent_dict["Symbol"].append(k)
    sent_dict["Score"].append(v["sentiment_score"])
    try:
        sent_dict["Name"].append(expansive_emoji_dict[k].replace(":", ""))
    except:
        sent_dict["Name"].append(np.nan)

In [113]:
available_sents_df = pd.DataFrame(sent_dict).dropna().reset_index(drop=True)

In [114]:
available_sents_df.head(20)

,Symbol,Score,Name
0,😂,0.221,face_with_tears_of_joy
1,❤,0.746,red_heart
2,♥,0.658,heart_suit
3,😍,0.678,smiling_face_with_heart-eyes
4,😭,-0.093,loudly_crying_face
5,😘,0.702,face_blowing_a_kiss
6,😊,0.645,smiling_face_with_smiling_eyes
7,👌,0.564,OK_hand
8,💕,0.633,two_hearts
9,👏,0.521,clapping_hands


In [119]:
available_sents_df.sort_values("Name").reset_index(drop=True, inplace=True)

In [127]:
available_sents_df=available_sents_df.sort_values("Name").reset_index(drop=True)

In [128]:
available_sents_df.loc[[175, 227, 337, 377, 403, 404, 514, 572], :]

,Symbol,Score,Name
175,😢,0.007,crying_face
227,😤,-0.211,face_with_steam_from_nose
337,💯,0.121,hundred_points
377,😭,-0.093,loudly_crying_face
403,💰,0.256,money_bag
404,💸,0.167,money_with_wings
514,😡,-0.173,pouting_face
572,🚀,0.525,rocket


In [129]:
available_sents_df.loc[175, "Score"] = -.45
available_sents_df.loc[227, "Score"] = -.405
available_sents_df.loc[337, "Score"] = .635
available_sents_df.loc[377, "Score"] = -.323
available_sents_df.loc[403, "Score"] = .596
available_sents_df.loc[404, "Score"] = .500
available_sents_df.loc[514, "Score"] = -.404
available_sents_df.loc[572, "Score"] = .645

In [144]:
available_sents_df.loc[753, "Score"] = .302

In [131]:
sia = sentiment.SentimentIntensityAnalyzer()

In [132]:
lexicon = sia.lexicon

In [135]:
lex_df = pd.DataFrame({"token": lexicon.keys(),
             "values": lexicon.values()})#.plot(kind="hist", y="values")

In [137]:
lex_df.sort_values("values").tail(15)

,token,values
2634,euphoric,3.2
4448,lovingly,3.2
4983,paradise,3.2
3285,glee,3.2
4439,love,3.2
3308,glorious,3.2
2492,elated,3.2
2470,ecstacy,3.3
6539,sweetheart,3.3
2633,euphoria,3.3


In [151]:
lex_df.sort_values("values").head(35)

,token,values
5465,rapist,-3.9
6092,slavery,-3.8
5464,raping,-3.8
4254,kill,-3.7
4720,murder,-3.7
238,fu,-3.7
5458,rape,-3.7
6644,terrorist,-3.7
4724,murderer,-3.6
5461,rapers,-3.6


In [152]:
# emojis should straddle the line of -3.5 and 3.2 so as to not overly index them to the poles of English Words extremes (bad emojis equivalent to cursing)
def lexnormScores(score):
    if score > 0:
        return round(score * 3.2, 1)
    else:
        return round(score * 3.4, 1)

In [153]:
available_sents_df = available_sents_df.assign(lexScore = lambda x: x.Score.apply(lexnormScores))

In [154]:
def lexify(name):
    if "_" in name:
        return name.lower()
    else:
        return "_" + name.lower()

In [155]:
available_sents_df = available_sents_df.assign(lexName = lambda x: x.Name.apply(lexify))

In [156]:
available_sents_df

,Symbol,Score,Name,lexScore,lexName
0,🏧,1.000,ATM_sign,3.2,atm_sign
1,🅰,0.333,A_button_(blood_type),1.1,a_button_(blood_type)
2,♈,-1.000,Aries,-3.4,_aries
3,🔙,0.217,BACK_arrow,0.7,back_arrow
4,🅱,0.000,B_button_(blood_type),0.0,b_button_(blood_type)
...,...,...,...,...,...
751,✍,1.000,writing_hand,3.2,writing_hand
752,💛,0.713,yellow_heart,2.3,yellow_heart
753,💴,0.302,yen_banknote,1.0,yen_banknote
754,☯,0.001,yin_yang,0.0,yin_yang


<h1 style="color: green; text-align: center">Process Flow</h1>
<ul style="text-align: left">
    <li style="color: green">Get Emoji Sentiment Where I can From Emosent module (replace spaces with _ ) -> enter into Vader lexicon as lowercase to keep upcase polarity diff that comes with VADER</li>
    <li style="color: green">Find Missing Emojis that are referenced in our sample: Enter them into the lexicon based on apparent sentiment of surrounding comment/context</li>
    <li style="color: green">Remove stop words from my Mentioned Stocks catcher</li>
    <li style="color: green">Show distribution of comments before emoji breakdown and after emoji breakdown</li>
    <li style="color: green">Move on to the most common words in our sample. Improve the OOTB Vader SIA to the enviroment of our current messages --> ENTER NEW WORDS INTO VADER LEXICON</li>
    <li style="color: green">In the end from our sample, we would like sentiment scores of comment (compound polarity scores), mentioned stocks (polarity of comments and amount of mentions) which can be integrated into our updatable dashboard that will help traders understand the private chat enviroment amongst traded equities, so that they can use this information as another tool in their toolbelt for their trading decisions.</li>
    <ul>

In [4]:
conn = sqlite3.connect("data/discord/discord.db")

In [5]:
comments_df = pd.read_sql("SELECT * FROM DiscordCommentsThursdayData", con=conn)

In [6]:
conn.close()

In [7]:
all_emojis_dict = emoji.UNICODE_EMOJI

In [8]:
emoji_sent_dict = emosent.EMOJI_SENTIMENT_DICT

In [17]:
# Ticker Info
nasdaqtickers = pd.read_csv("./data/listedCompanies/nasdaqtraded.txt", sep="|")
moretickers = pd.read_csv("./data/listedCompanies/otherlisted.txt", sep="|")

In [50]:
nasdaqtickers.groupby("Listing Exchange").count()

,Nasdaq Traded,Symbol,Security Name,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Listing Exchange,,,,,,,,,,,
A,298,298,298,298,298,298,298,0,298,298,298
N,3280,3280,3280,3280,3280,3280,3280,0,3280,3280,3280
P,1554,1554,1554,1554,1554,1554,1554,0,1554,1554,1554
Q,3851,3851,3851,3851,3851,3851,3851,3851,0,3851,3851
V,3,3,3,3,3,3,3,0,3,3,3
Z,418,418,418,418,418,418,418,0,418,418,418


In [52]:
moretickers.groupby("Exchange").count()

,ACT Symbol,Security Name,CQS Symbol,ETF,Round Lot Size,Test Issue,NASDAQ Symbol
Exchange,,,,,,,
A,298,298,298,298,298,298,298
N,3280,3280,3280,3280,3280,3280,3280
P,1554,1554,1554,1554,1554,1554,1554
V,3,3,3,3,3,3,3
Z,418,418,418,418,418,418,418


In [56]:
nasdaqtickers[nasdaqtickers["Symbol"] != nasdaqtickers["NASDAQ Symbol"]]

,Nasdaq Traded,Symbol,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
10,Y,AAIC$B,Arlington Asset Investment Corp 7.00%,N,,N,100,N,NaN,AAICpB,AAIC-B,N
11,Y,AAIC$C,Arlington Asset Investment Corp 8.250% Seies C...,N,,N,100,N,NaN,AAICpC,AAIC-C,N
39,Y,ABR$A,Arbor Realty Trust Preferred Series A,N,,N,100,N,NaN,ABRpA,ABR-A,N
40,Y,ABR$B,Arbor Realty Trust Cumulative Redeemable Prefe...,N,,N,100,N,NaN,ABRpB,ABR-B,N
41,Y,ABR$C,Arbor Realty Trust Cumulative Redeemable Prefe...,N,,N,100,N,NaN,ABRpC,ABR-C,N
...,...,...,...,...,...,...,...,...,...,...,...,...
9208,Y,XAN$C,Exantas Capital Corp.,N,,N,100,N,NaN,XANpC,XAN-C,N
9273,Y,XPOA.U,"DPCM Capital, Inc. Units, each consisting of o...",N,,N,100,N,NaN,XPOA.U,XPOA=,N
9303,Y,YAC.U,"Yucaipa Acquisition Corporation Units, each co...",N,,N,100,N,NaN,YAC.U,YAC=,N
9304,Y,YAC.W,Yucaipa Acquisition Corporation Redeemable War...,N,,N,100,N,NaN,YAC.WS,YAC+,N


In [58]:
tickerInfo = nasdaqtickers[nasdaqtickers["Test Issue"] != "Y"]

In [59]:
tickerInfo.shape

(9370, 12)

In [81]:
conn = sqlite3.connect("data/discord/text.db")
comments_df2 = pd.read_sql("SELECT * FROM text_588270137874841620_588284257491877889", con=conn) # original data
conn.close()

In [82]:
comments_df2

,id,name,content,timestamp
0,488359813160501251,Merente#5844,Thanks <@464449226521313292> ! I opted for th...,2020-11-30T20:48:43.035000+00:00
1,689681229989150742,jackaplan#7289,Barely followed the market at all today (didn’...,2020-11-30T20:58:32.872000+00:00
2,612337363326074936,Trademate#2515,<@131952052069728258> has reached level **13**!,2020-11-30T19:55:20.913000+00:00
3,488359813160501251,Merente#5844,Thanks <@464449226521313292> ! I opted for th...,2020-11-30T20:48:43.035000+00:00
4,689681229989150742,jackaplan#7289,Barely followed the market at all today (didn’...,2020-11-30T20:58:32.872000+00:00
...,...,...,...,...
38255,579205396166737921,beenpippinsincebeenpippin#4039,@everyone today’s total. This doesn’t calculat...,2019-08-02T01:13:47.923000+00:00
38256,579205396166737921,beenpippinsincebeenpippin#4039,Play account today,2019-08-02T01:10:52.637000+00:00
38257,385261276986605568,Sporadic#0001,@everyone,2019-08-02T01:11:08.694000+00:00
38258,579205396166737921,beenpippinsincebeenpippin#4039,Play account today,2019-08-02T01:10:52.637000+00:00


In [83]:
comments_df2=comments_df2.drop_duplicates()

In [84]:
comments_df2=comments_df2.assign(Datetime = lambda x: x.timestamp.apply(datetime.datetime.fromisoformat))

In [72]:
def easternize(dt):
    return dt.tz_convert("est")

In [85]:
def timeofday(h):
    return h.hour + (h.minute / 60)

In [86]:
comments_df2 = comments_df2.assign(EasternDT = lambda x: x.Datetime.apply(easternize))

In [87]:
comments_df2  = comments_df2.assign(Hours = lambda x: x.EasternDT.apply(timeofday))

In [88]:
comments_df2

,id,name,content,timestamp,Datetime,EasternDT,Hours
0,488359813160501251,Merente#5844,Thanks <@464449226521313292> ! I opted for th...,2020-11-30T20:48:43.035000+00:00,2020-11-30 20:48:43.035000+00:00,2020-11-30 15:48:43.035000-05:00,15.800000
1,689681229989150742,jackaplan#7289,Barely followed the market at all today (didn’...,2020-11-30T20:58:32.872000+00:00,2020-11-30 20:58:32.872000+00:00,2020-11-30 15:58:32.872000-05:00,15.966667
2,612337363326074936,Trademate#2515,<@131952052069728258> has reached level **13**!,2020-11-30T19:55:20.913000+00:00,2020-11-30 19:55:20.913000+00:00,2020-11-30 14:55:20.913000-05:00,14.916667
5,131952052069728258,Teecee#3022,<@!464449226521313292> thanks boss,2020-11-30T19:55:20.625000+00:00,2020-11-30 19:55:20.625000+00:00,2020-11-30 14:55:20.625000-05:00,14.916667
9,464449226521313292,TradingJEDI#2647,<@!399198468159045633> BRO AWSOME SO GLAD YOU ...,2020-11-30T19:44:26.516000+00:00,2020-11-30 19:44:26.516000+00:00,2020-11-30 14:44:26.516000-05:00,14.733333
...,...,...,...,...,...,...,...
38226,445583268792893449,itsyoboyvic#7198,Big crack,2019-08-02T01:14:39.682000+00:00,2019-08-02 01:14:39.682000+00:00,2019-08-01 20:14:39.682000-05:00,20.233333
38231,445583268792893449,itsyoboyvic#7198,Crack,2019-08-02T01:14:35.798000+00:00,2019-08-02 01:14:35.798000+00:00,2019-08-01 20:14:35.798000-05:00,20.233333
38236,579205396166737921,beenpippinsincebeenpippin#4039,@everyone today’s total. This doesn’t calculat...,2019-08-02T01:13:47.923000+00:00,2019-08-02 01:13:47.923000+00:00,2019-08-01 20:13:47.923000-05:00,20.216667
38245,385261276986605568,Sporadic#0001,@everyone,2019-08-02T01:11:08.694000+00:00,2019-08-02 01:11:08.694000+00:00,2019-08-01 20:11:08.694000-05:00,20.183333


In [91]:
# Bot congrats
botCom = re.compile("\shas reached level \*\*.\d+\*\*")
bot2Com = re.compile("Congrats \<.*\>, you just advanced to")

In [92]:
def catchBot(c):
    botreg = botCom.search(c)
    botreg1 = bot2Com.search(c.strip())
    if (botreg != None) or (botreg1 != None):
        return 1
    else:
        return 0

In [94]:
comments_df2=comments_df2.assign(IsBot = lambda x: x.content.apply(catchBot))

In [95]:
comments_df2.groupby("IsBot").count()

,id,name,content,timestamp,Datetime,EasternDT,Hours
IsBot,,,,,,,
Bot,52,52,52,52,52,52,52
Human,5029,5029,5029,5029,5029,5029,5029


In [159]:
real_comments_df = comments_df2[comments_df2.IsBot == "Human"].reset_index(drop=True)

In [160]:
real_comments_df

,id,name,content,timestamp,Datetime,EasternDT,Hours,IsBot
0,488359813160501251,Merente#5844,Thanks <@464449226521313292> ! I opted for th...,2020-11-30T20:48:43.035000+00:00,2020-11-30 20:48:43.035000+00:00,2020-11-30 15:48:43.035000-05:00,15.800000,Human
1,689681229989150742,jackaplan#7289,Barely followed the market at all today (didn’...,2020-11-30T20:58:32.872000+00:00,2020-11-30 20:58:32.872000+00:00,2020-11-30 15:58:32.872000-05:00,15.966667,Human
2,131952052069728258,Teecee#3022,<@!464449226521313292> thanks boss,2020-11-30T19:55:20.625000+00:00,2020-11-30 19:55:20.625000+00:00,2020-11-30 14:55:20.625000-05:00,14.916667,Human
3,464449226521313292,TradingJEDI#2647,<@!399198468159045633> BRO AWSOME SO GLAD YOU ...,2020-11-30T19:44:26.516000+00:00,2020-11-30 19:44:26.516000+00:00,2020-11-30 14:44:26.516000-05:00,14.733333,Human
4,399198468159045633,AjKilgore#0424,<@!464449226521313292> might have sold a bit ...,2020-11-30T19:30:11.004000+00:00,2020-11-30 19:30:11.004000+00:00,2020-11-30 14:30:11.004000-05:00,14.500000,Human
...,...,...,...,...,...,...,...,...
5024,445583268792893449,itsyoboyvic#7198,Big crack,2019-08-02T01:14:39.682000+00:00,2019-08-02 01:14:39.682000+00:00,2019-08-01 20:14:39.682000-05:00,20.233333,Human
5025,445583268792893449,itsyoboyvic#7198,Crack,2019-08-02T01:14:35.798000+00:00,2019-08-02 01:14:35.798000+00:00,2019-08-01 20:14:35.798000-05:00,20.233333,Human
5026,579205396166737921,beenpippinsincebeenpippin#4039,@everyone today’s total. This doesn’t calculat...,2019-08-02T01:13:47.923000+00:00,2019-08-02 01:13:47.923000+00:00,2019-08-01 20:13:47.923000-05:00,20.216667,Human
5027,385261276986605568,Sporadic#0001,@everyone,2019-08-02T01:11:08.694000+00:00,2019-08-02 01:11:08.694000+00:00,2019-08-01 20:11:08.694000-05:00,20.183333,Human


In [162]:
comments = real_comments_df.content.values

In [163]:
comments

array(['Thanks <@464449226521313292> !  I opted for the protection on this which limited my profits but not much.  Still a 30% trade for me!',
       'Barely followed the market at all today (didn’t like the price action) but I’ll take $160 any day',
       '<@!464449226521313292> thanks boss', ...,
       '@everyone today’s total. This doesn’t calculate the after hours earnings strangles. They will be included in tomorrow’s profit.',
       '@everyone', 'Play account today'], dtype=object)

In [222]:
used_emotes = {}
needed_Emotes = []
for com in comments:
    emoji_catcher = emoji.emoji_lis(com)
    if emoji_catcher != []:
        for e in emoji_catcher:
            name = expansive_emoji_dict[e["emoji"]]
            if ("skin_tone" in name) or ("regional_indicator_" in name):
                    next
            elif (":male_sign:" == name) or (":female_sign:" == name):
                    next
            else:
                if e["emoji"] not in used_emotes.keys():
                    used_emotes[e["emoji"]] = {}
                    used_emotes[e["emoji"]]["value"] = 1
                    used_emotes[e["emoji"]]["name"] = name
                    if e["emoji"] in emosent_emoj_dict.keys():
                        used_emotes[e["emoji"]]["hasScore"] = True
                        used_emotes[e["emoji"]]["prospectiveScore"] = emosent_emoj_dict[e["emoji"]]["score"]
                    else:
                        used_emotes[e["emoji"]]["hasScore"] = False
                        needed_Emotes.append(e["emoji"])
                else:
                    used_emotes[e["emoji"]]["value"] += 1
                    used_emotes[e["emoji"]]["name"] = name
        

In [221]:
used_emotes

{'💯': {'value': 24,
  'name': ':hundred_points:',
  'hasScore': True,
  'prospectiveScore': 2.0},
 '🤞': {'value': 1, 'name': ':crossed_fingers:', 'hasScore': False},
 '🤣': {'value': 9,
  'name': ':rolling_on_the_floor_laughing:',
  'hasScore': False},
 '👇': {'value': 18,
  'name': ':backhand_index_pointing_down:',
  'hasScore': True,
  'prospectiveScore': 0.8},
 '😭': {'value': 6,
  'name': ':loudly_crying_face:',
  'hasScore': True,
  'prospectiveScore': -1.1},
 '👍': {'value': 33,
  'name': ':thumbs_up:',
  'hasScore': True,
  'prospectiveScore': 1.7},
 '🙏': {'value': 51,
  'name': ':folded_hands:',
  'hasScore': True,
  'prospectiveScore': 1.3},
 '😉': {'value': 23,
  'name': ':winking_face:',
  'hasScore': True,
  'prospectiveScore': 1.5},
 '🙂': {'value': 24, 'name': ':slightly_smiling_face:', 'hasScore': False},
 '👀': {'value': 11,
  'name': ':eyes:',
  'hasScore': True,
  'prospectiveScore': 0.2},
 '😂': {'value': 50,
  'name': ':face_with_tears_of_joy:',
  'hasScore': True,
  'prosp

In [208]:
as_df = available_sents_df.loc[:, ["Symbol", "lexScore", "lexName"]]

In [211]:
emosent_emoj = as_df.Symbol.values

In [216]:
scores = as_df.lexScore.values

In [215]:
names = as_df.lexName.values

In [217]:
emosent_emoj_dict = {}
for i, e in enumerate(emosent_emoj):
    emosent_emoj_dict[e] = {}
    emosent_emoj_dict[e]["score"] = scores[i]
    emosent_emoj_dict[e]["lexname"] = names[i]

In [218]:
emosent_emoj_dict

{'🏧': {'score': 3.2, 'lexname': 'atm_sign'},
 '🅰': {'score': 1.1, 'lexname': 'a_button_(blood_type)'},
 '♈': {'score': -3.4, 'lexname': '_aries'},
 '🔙': {'score': 0.7, 'lexname': 'back_arrow'},
 '🅱': {'score': 0.0, 'lexname': 'b_button_(blood_type)'},
 '🆑': {'score': 0.0, 'lexname': 'cl_button'},
 '🆒': {'score': 1.3, 'lexname': 'cool_button'},
 '♋': {'score': 1.1, 'lexname': '_cancer'},
 '🎄': {'score': 1.7, 'lexname': 'christmas_tree'},
 '🔚': {'score': 1.4, 'lexname': 'end_arrow'},
 '🆓': {'score': 0.8, 'lexname': 'free_button'},
 '♊': {'score': 0.0, 'lexname': '_gemini'},
 '🆔': {'score': 3.0, 'lexname': 'id_button'},
 '🉐': {'score': 3.2, 'lexname': 'japanese_bargain_button'},
 '🏯': {'score': 0.5, 'lexname': 'japanese_castle'},
 '🈹': {'score': 1.1, 'lexname': 'japanese_discount_button'},
 '🎎': {'score': 3.1, 'lexname': 'japanese_dolls'},
 '🈂': {'score': -3.4, 'lexname': 'japanese_service_charge_button'},
 '🔰': {'score': 0.0, 'lexname': 'japanese_symbol_for_beginner'},
 '♌': {'score': 2.

In [223]:
needed_Emotes

['🤞',
 '🤣',
 '🙂',
 '🤩',
 '🤑',
 '🤠',
 '🥳',
 '🛫',
 '🤦',
 '🤫',
 '\U0001f9be',
 '🥰',
 '🤔',
 '🤙',
 '🧠',
 '🤝',
 '🧐',
 '🤧',
 '🥺',
 '\U0001f9c3',
 '🤤',
 '🤬',
 '🤯',
 '🤗',
 '🛥',
 '🖐',
 '🤷',
 '🕵',
 '🤘',
 '🦕']

In [227]:
emComm_comments = []
for c in comments:
    elist = emoji.emoji_lis(c)
    for e in elist:
        if e["emoji"] in needed_Emotes:
            emComm_comments.append(c)

In [228]:
emComm_comments

['<@703659646279483503> <@!579205396166737921> 💯🤞🏾',
 '<@579205396166737921> not sure why someone bought one of my ACB contracts for 3.7 but I bought it back for $52 less a few minutes later 🤣',
 '🙂',
 "> <@!629362548919107584> damn. you're insane with that growth!\n<@!680245219706667048> i had a bit of luck on my side 🙂",
 'baby gains 🙂',
 '🤩',
 'Unity 🤑',
 'yessir another one. I do it for trademate ig 🤑🤑',
 'yessir another one. I do it for trademate ig 🤑🤑',
 'Finally some green🤠🤠 Just got enough money for another month here on TM <:zuckstocksup:756648506047070259> \nThanks to <@!579205396166737921>',
 'Finally some green🤠🤠 Just got enough money for another month here on TM <:zuckstocksup:756648506047070259> \nThanks to <@!579205396166737921>',
 'Small acc challenge 🤑',
 '<@!579205396166737921> I’ve got a challenge, turn my 2.5k account into 10k 🤣',
 'page bouta be flooded w fat snap gains tmo 🤑🤑',
 'page bouta be flooded w fat snap gains tmo 🤑🤑',
 'TP too early and still walked away 

In [229]:
emoji_lex2 = {}
for n in needed_Emotes:
    name = expansive_emoji_dict[n].replace(":", "")
    if "_" not in name:
        name = "_" + name
    emoji_lex2[n] = {}
    emoji_lex2[n]["lexName"] = name

In [230]:
emoji_lex2

{'🤞': {'lexName': 'crossed_fingers'},
 '🤣': {'lexName': 'rolling_on_the_floor_laughing'},
 '🙂': {'lexName': 'slightly_smiling_face'},
 '🤩': {'lexName': '_star-struck'},
 '🤑': {'lexName': 'money-mouth_face'},
 '🤠': {'lexName': 'cowboy_hat_face'},
 '🥳': {'lexName': 'partying_face'},
 '🛫': {'lexName': 'airplane_departure'},
 '🤦': {'lexName': 'person_facepalming'},
 '🤫': {'lexName': 'shushing_face'},
 '\U0001f9be': {'lexName': 'mechanical_arm'},
 '🥰': {'lexName': 'smiling_face_with_3_hearts'},
 '🤔': {'lexName': 'thinking_face'},
 '🤙': {'lexName': 'call_me_hand'},
 '🧠': {'lexName': '_brain'},
 '🤝': {'lexName': '_handshake'},
 '🧐': {'lexName': 'face_with_monocle'},
 '🤧': {'lexName': 'sneezing_face'},
 '🥺': {'lexName': 'pleading_face'},
 '\U0001f9c3': {'lexName': 'beverage_box'},
 '🤤': {'lexName': 'drooling_face'},
 '🤬': {'lexName': 'face_with_symbols_on_mouth'},
 '🤯': {'lexName': 'exploding_head'},
 '🤗': {'lexName': 'hugging_face'},
 '🛥': {'lexName': 'motor_boat'},
 '🖐': {'lexName': 'hand_wi

In [254]:
emoji_lex2 = {'🤞': {'lexName': 'crossed_fingers', "value": 0.4},
 '🤣': {'lexName': 'rolling_on_the_floor_laughing', "value": 2.7},
 '🙂': {'lexName': 'slightly_smiling_face', "value": 1.0},
 '🤩': {'lexName': '_star-struck', "value": 1.4},
 '🤑': {'lexName': 'money-mouth_face', "value": 2.5},
 '🤠': {'lexName': 'cowboy_hat_face', "value": 2.1},
 '🥳': {'lexName': 'partying_face', "value": 1.7},
 '🛫': {'lexName': 'airplane_departure', "value": 1.5},
 '🤦': {'lexName': 'person_facepalming', "value":-1.5},
 '🤫': {'lexName': 'shushing_face', "value": 0.3},
 '\U0001f9be': {'lexName': 'mechanical_arm', "value": 1.8},
 '🥰': {'lexName': 'smiling_face_with_3_hearts', "value": 2.3},
 '🤔': {'lexName': 'thinking_face', "value": 0.1},
 '🤙': {'lexName': 'call_me_hand', "value": 1.3},
 '🧠': {'lexName': '_brain', "value": 1.7},
 '🤝': {'lexName': '_handshake', "value": 1.5},
 '🧐': {'lexName': 'face_with_monocle', "value": 1.7},
 '🤧': {'lexName': 'sneezing_face', "value": -1.2},
 '🥺': {'lexName': 'pleading_face', "value": -0.5},
 '\U0001f9c3': {'lexName': 'beverage_box', "value": 2.0},
 '🤤': {'lexName': 'drooling_face', "value": 1.2},
 '🤬': {'lexName': 'face_with_symbols_on_mouth', "value": -2.2},
 '🤯': {'lexName': 'exploding_head', "value": .6},
 '🤗': {'lexName': 'hugging_face', "value": 2.0},
 '🛥': {'lexName': 'motor_boat', "value": 2.1},
 '🖐': {'lexName': 'hand_with_fingers_splayed', "value": 0.5},
 '🤷': {'lexName': 'person_shrugging', "value": -0.4},
 '🕵': {'lexName': '_detective', "value": 1.3},
 '🤘': {'lexName': 'sign_of_the_horns', "value": 1.3},
 '🦕': {'lexName': '_sauropod', "value": 1.5}
}

In [253]:
lexicon["idk"]

-0.4

In [244]:
used_emotes

{'💯': {'value': 24,
  'name': ':hundred_points:',
  'hasScore': True,
  'prospectiveScore': 2.0},
 '🤞': {'value': 1, 'name': ':crossed_fingers:', 'hasScore': False},
 '🤣': {'value': 9,
  'name': ':rolling_on_the_floor_laughing:',
  'hasScore': False},
 '👇': {'value': 18,
  'name': ':backhand_index_pointing_down:',
  'hasScore': True,
  'prospectiveScore': 0.8},
 '😭': {'value': 6,
  'name': ':loudly_crying_face:',
  'hasScore': True,
  'prospectiveScore': -1.1},
 '👍': {'value': 33,
  'name': ':thumbs_up:',
  'hasScore': True,
  'prospectiveScore': 1.7},
 '🙏': {'value': 51,
  'name': ':folded_hands:',
  'hasScore': True,
  'prospectiveScore': 1.3},
 '😉': {'value': 23,
  'name': ':winking_face:',
  'hasScore': True,
  'prospectiveScore': 1.5},
 '🙂': {'value': 24, 'name': ':slightly_smiling_face:', 'hasScore': False},
 '👀': {'value': 11,
  'name': ':eyes:',
  'hasScore': True,
  'prospectiveScore': 0.2},
 '😂': {'value': 50,
  'name': ':face_with_tears_of_joy:',
  'hasScore': True,
  'prosp

In [256]:
emosent_emoj_dict

{'🏧': {'score': 3.2, 'lexname': 'atm_sign'},
 '🅰': {'score': 1.1, 'lexname': 'a_button_(blood_type)'},
 '♈': {'score': -3.4, 'lexname': '_aries'},
 '🔙': {'score': 0.7, 'lexname': 'back_arrow'},
 '🅱': {'score': 0.0, 'lexname': 'b_button_(blood_type)'},
 '🆑': {'score': 0.0, 'lexname': 'cl_button'},
 '🆒': {'score': 1.3, 'lexname': 'cool_button'},
 '♋': {'score': 1.1, 'lexname': '_cancer'},
 '🎄': {'score': 1.7, 'lexname': 'christmas_tree'},
 '🔚': {'score': 1.4, 'lexname': 'end_arrow'},
 '🆓': {'score': 0.8, 'lexname': 'free_button'},
 '♊': {'score': 0.0, 'lexname': '_gemini'},
 '🆔': {'score': 3.0, 'lexname': 'id_button'},
 '🉐': {'score': 3.2, 'lexname': 'japanese_bargain_button'},
 '🏯': {'score': 0.5, 'lexname': 'japanese_castle'},
 '🈹': {'score': 1.1, 'lexname': 'japanese_discount_button'},
 '🎎': {'score': 3.1, 'lexname': 'japanese_dolls'},
 '🈂': {'score': -3.4, 'lexname': 'japanese_service_charge_button'},
 '🔰': {'score': 0.0, 'lexname': 'japanese_symbol_for_beginner'},
 '♌': {'score': 2.

In [257]:
emoji_lex2

{'🤞': {'lexName': 'crossed_fingers', 'value': 0.4},
 '🤣': {'lexName': 'rolling_on_the_floor_laughing', 'value': 2.7},
 '🙂': {'lexName': 'slightly_smiling_face', 'value': 1.0},
 '🤩': {'lexName': '_star-struck', 'value': 1.4},
 '🤑': {'lexName': 'money-mouth_face', 'value': 2.5},
 '🤠': {'lexName': 'cowboy_hat_face', 'value': 2.1},
 '🥳': {'lexName': 'partying_face', 'value': 1.7},
 '🛫': {'lexName': 'airplane_departure', 'value': 1.5},
 '🤦': {'lexName': 'person_facepalming', 'value': -1.5},
 '🤫': {'lexName': 'shushing_face', 'value': 0.3},
 '\U0001f9be': {'lexName': 'mechanical_arm', 'value': 1.8},
 '🥰': {'lexName': 'smiling_face_with_3_hearts', 'value': 2.3},
 '🤔': {'lexName': 'thinking_face', 'value': 0.1},
 '🤙': {'lexName': 'call_me_hand', 'value': 1.3},
 '🧠': {'lexName': '_brain', 'value': 1.7},
 '🤝': {'lexName': '_handshake', 'value': 1.5},
 '🧐': {'lexName': 'face_with_monocle', 'value': 1.7},
 '🤧': {'lexName': 'sneezing_face', 'value': -1.2},
 '🥺': {'lexName': 'pleading_face', 'value'

In [259]:
emosent_emoj_dict["💯"]

{'score': 2.0, 'lexname': 'hundred_points'}

In [260]:
emote_codes = {}
for k in emosent_emoj_dict:
    lex = emosent_emoj_dict[k]["lexname"] # lex_dict name
    score = emosent_emoj_dict[k]["score"]
    emote_codes[lex] = score

In [264]:
for k in emoji_lex2:
    lex = emoji_lex2[k]["lexName"] # lex_dict name
    score = emoji_lex2[k]["value"]
    emote_codes[lex] = score

In [266]:
lexicon.update(emote_codes)

In [276]:
tickerInfo[tickerInfo.Symbol != tickerInfo["NASDAQ Symbol"]]

,Nasdaq Traded,Symbol,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
10,Y,AAIC$B,Arlington Asset Investment Corp 7.00%,N,,N,100,N,NaN,AAICpB,AAIC-B,N
11,Y,AAIC$C,Arlington Asset Investment Corp 8.250% Seies C...,N,,N,100,N,NaN,AAICpC,AAIC-C,N
39,Y,ABR$A,Arbor Realty Trust Preferred Series A,N,,N,100,N,NaN,ABRpA,ABR-A,N
40,Y,ABR$B,Arbor Realty Trust Cumulative Redeemable Prefe...,N,,N,100,N,NaN,ABRpB,ABR-B,N
41,Y,ABR$C,Arbor Realty Trust Cumulative Redeemable Prefe...,N,,N,100,N,NaN,ABRpC,ABR-C,N
...,...,...,...,...,...,...,...,...,...,...,...,...
9208,Y,XAN$C,Exantas Capital Corp.,N,,N,100,N,NaN,XANpC,XAN-C,N
9273,Y,XPOA.U,"DPCM Capital, Inc. Units, each consisting of o...",N,,N,100,N,NaN,XPOA.U,XPOA=,N
9303,Y,YAC.U,"Yucaipa Acquisition Corporation Units, each co...",N,,N,100,N,NaN,YAC.U,YAC=,N
9304,Y,YAC.W,Yucaipa Acquisition Corporation Redeemable War...,N,,N,100,N,NaN,YAC.WS,YAC+,N


In [277]:
ticker_translater = {}
for i, r in tickerInfo.iterrows():
    if r["Symbol"] != r["NASDAQ Symbol"]:
        sym = r["Symbol"].lower()
        nas_sym = r["NASDAQ Symbol"].lower()
        cqs_sym = r["CQS Symbol"].lower()
        ticker_translater[sym] = r["Security Name"]
        ticker_translater[nas_sym] = r["Security Name"]
        ticker_translater[cqs_sym] = r["Security Name"]
    else:
        sym = r["Symbol"].lower()
        ticker_translater[sym] = r["Security Name"]

In [330]:
!pwd

/c/Users/Kyle/Desktop/stockProject/demo-project/NLP


In [283]:
expansive_emoji_dict['🤣'].replace(":", "")

'rolling_on_the_floor_laughing'

In [1165]:
def commentFactory(com):
    comment = com
    comment = comment.replace("\n", "")
    # handle mentions
    mention_pattern = "<[0-9!#&]*>"
    mentions = re.findall(mention_pattern, comment)
    if mentions != []: # replace mentions
        for m in mention:
            comment = com.replace(m, "")
    emoji_list = emoji.emoji_lis(comment)
    # Handled Emotes: translate to their lexical representation
    if emoji_list != []:
        for emote in emoji_list:
            code = expansive_emoji_dict[emote["emoji"]]
            if ("skin_tone" in code) or ("regional_indicator_" in code):
                    next
            elif (":male_sign:" == code) or (":female_sign:" == code):
                    next
            code = code.replace(":", " ") # 'unstick' emotes
            comment = comment.replace(emote, code)
#             if code not in emote_codes.keys(): # deal with unhandled emote
#                 if "_" not in code:
#                     code = "_" + code # single word emoji
#                 comment = comment.remove(code, "")
#                 sia.polarity_scores(comment)[""]
    sentiment_dict = sia.polarity_scores(comment)
    scores_l = [sentiment_dict["neg"], sentiment_dict["neu"], sentiment_dict["pos"],  sentiment_dict["compound"]]
    return ", ".join([s for s in scores_l])

In [272]:
re.findall("<[@0-9!#]*>", comments[0])

['<@464449226521313292>']

In [309]:
comments[22]

'<@579205396166737921> not sure why someone bought one of my ACB contracts for 3.7 but I bought it back for $52 less a few minutes later 🤣'

In [310]:
ssample = comments[22]

In [477]:
mention_pattern = "<[@0-9!&#]*>"
mentions = re.findall(mention_pattern, ssample)
for m in mentions:
    ssample = ssample.replace(m, "")
emojis_ll = emoji.emoji_lis(ssample)
for e in emojis_ll:
    codex = expansive_emoji_dict[e["emoji"]].replace(":", "")
    ssample = ssample.replace(e["emoji"], codex) 
print(ssample)

 not sure why someone bought one of my ACB contracts for 3.7 but I bought it back for $52 less a few minutes later rolling_on_the_floor_laughing


In [314]:
sia.polarity_scores(comments[22])

{'neg': 0.063, 'neu': 0.937, 'pos': 0.0, 'compound': -0.1232}

In [315]:
# WITH EMOJI REPLACEMENT
sia.polarity_scores(ssample)

{'neg': 0.054, 'neu': 0.763, 'pos': 0.183, 'compound': 0.6777}

In [318]:
comments[22]

'<@579205396166737921> not sure why someone bought one of my ACB contracts for 3.7 but I bought it back for $52 less a few minutes later 🤣'

In [321]:
strings = "this is a sazzze sentence"

In [328]:
word_tokenize(comments[444])

['small', '$', 'gain', 'but', 'decent', '%', 'gain', '😂', 'all', 'adds', 'up']

In [329]:
comments[444]

'small $ gain but decent % gain 😂 all adds up'

In [331]:
emComm_comments

['<@703659646279483503> <@!579205396166737921> 💯🤞🏾',
 '<@579205396166737921> not sure why someone bought one of my ACB contracts for 3.7 but I bought it back for $52 less a few minutes later 🤣',
 '🙂',
 "> <@!629362548919107584> damn. you're insane with that growth!\n<@!680245219706667048> i had a bit of luck on my side 🙂",
 'baby gains 🙂',
 '🤩',
 'Unity 🤑',
 'yessir another one. I do it for trademate ig 🤑🤑',
 'yessir another one. I do it for trademate ig 🤑🤑',
 'Finally some green🤠🤠 Just got enough money for another month here on TM <:zuckstocksup:756648506047070259> \nThanks to <@!579205396166737921>',
 'Finally some green🤠🤠 Just got enough money for another month here on TM <:zuckstocksup:756648506047070259> \nThanks to <@!579205396166737921>',
 'Small acc challenge 🤑',
 '<@!579205396166737921> I’ve got a challenge, turn my 2.5k account into 10k 🤣',
 'page bouta be flooded w fat snap gains tmo 🤑🤑',
 'page bouta be flooded w fat snap gains tmo 🤑🤑',
 'TP too early and still walked away 

In [337]:
final_dataframe = comments_df2[comments_df2.IsBot != "Bot"]

In [339]:
final_dataframe = final_dataframe.reset_index(drop=True) 

In [340]:
final_dataframe

,id,name,content,timestamp,Datetime,EasternDT,Hours,IsBot
0,488359813160501251,Merente#5844,Thanks <@464449226521313292> ! I opted for th...,2020-11-30T20:48:43.035000+00:00,2020-11-30 20:48:43.035000+00:00,2020-11-30 15:48:43.035000-05:00,15.800000,Human
1,689681229989150742,jackaplan#7289,Barely followed the market at all today (didn’...,2020-11-30T20:58:32.872000+00:00,2020-11-30 20:58:32.872000+00:00,2020-11-30 15:58:32.872000-05:00,15.966667,Human
2,131952052069728258,Teecee#3022,<@!464449226521313292> thanks boss,2020-11-30T19:55:20.625000+00:00,2020-11-30 19:55:20.625000+00:00,2020-11-30 14:55:20.625000-05:00,14.916667,Human
3,464449226521313292,TradingJEDI#2647,<@!399198468159045633> BRO AWSOME SO GLAD YOU ...,2020-11-30T19:44:26.516000+00:00,2020-11-30 19:44:26.516000+00:00,2020-11-30 14:44:26.516000-05:00,14.733333,Human
4,399198468159045633,AjKilgore#0424,<@!464449226521313292> might have sold a bit ...,2020-11-30T19:30:11.004000+00:00,2020-11-30 19:30:11.004000+00:00,2020-11-30 14:30:11.004000-05:00,14.500000,Human
...,...,...,...,...,...,...,...,...
5024,445583268792893449,itsyoboyvic#7198,Big crack,2019-08-02T01:14:39.682000+00:00,2019-08-02 01:14:39.682000+00:00,2019-08-01 20:14:39.682000-05:00,20.233333,Human
5025,445583268792893449,itsyoboyvic#7198,Crack,2019-08-02T01:14:35.798000+00:00,2019-08-02 01:14:35.798000+00:00,2019-08-01 20:14:35.798000-05:00,20.233333,Human
5026,579205396166737921,beenpippinsincebeenpippin#4039,@everyone today’s total. This doesn’t calculat...,2019-08-02T01:13:47.923000+00:00,2019-08-02 01:13:47.923000+00:00,2019-08-01 20:13:47.923000-05:00,20.216667,Human
5027,385261276986605568,Sporadic#0001,@everyone,2019-08-02T01:11:08.694000+00:00,2019-08-02 01:11:08.694000+00:00,2019-08-01 20:11:08.694000-05:00,20.183333,Human


In [593]:
stop_words = set(stopwords.words("english"))

In [1186]:
additional_stop_words = set(["one", "good", "play", "nice", "see", "mind", "alex", "moon", "cane", "k",
   "go", "man", "big", "work", "want", "well", "son", "fast",  "mac", "cut", "pic", "rode", "itm",
   "cut", "love", "free", "next", "new", "post", "onto", "pdt", "tm", "f", "shy", "exp", "max", "cuz",
   "ago", "best", "job", "ever", "hope", "old", "cd", "min", "ath", "life", "leg", "gut", "ppl", "ngl", "bud", "bc", "leap",
   "away", "home", "low", "ty", "tho", "hope", "ta", "grow", "safe", "ten", "pack", "cry", "peak", "def",
   "pays", "fat", "oxy", "wow", "key", "race", "goat", "bk", "hr", "team", "lazy", "ha", "true", "pm", 
   "earn", "flex", "ride", "bit", "bro", "boss", "td", "boom", "ten", "save", "fill", "w", "l", "dd", 
    "term", "rh", "mine", "size", "step", "near", "cash", "pick", "eat", "two", "plan", "stay", "bout", "hi",
    "gain", "else", "huge", "fun", "glad", "mark", "plus", "eyes", "live", "c", "pump", "door", "auto",
                            "roll", "tell", "fam", "turn", "mine", "yolo", "real", "hold", "run", "u", "fast", "beat",
                            "sep", "oct", "nov", "dec", "jan", "feb", "mar", "apr", "may", "smh", "self", "phat", "j", "rock", "toke"])

In [1187]:
def findCompanies(comment):
    # lowercase the comment 
    m_comment = comment.lower()
    if "elon" in m_comment:
        m_comment = m_comment + " tsla"
    # remove all mentions
    pattern = "<.*>"
    mentions = re.findall(pattern, m_comment)
    if mentions != []:
        for m in mentions:
            m_comment = m_comment.replace(m, "")
    #remove all emojis
    emojiPattern = emoji.get_emoji_regexp()
    emojis = re.findall(emojiPattern, m_comment)
    if emojis != []:
        for emote in emojis:
            m_comment = m_comment.replace(emote, "")
    # remove all punctuations
    # m_comment.translate(str.maketrans('', '', string.punctuation))
    m_comment=m_comment.replace("\n", "")
    mentioned_companies = []
    sentences_given = sent_tokenize(m_comment)
    for sentence in sentences_given:
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))
        tokens = word_tokenize(sentence)
        for token in tokens:
            token = token.lower()
            if (token not in stop_words) and (token not in additional_stop_words) and (token in ticker_translater.keys()):
                mentioned_companies.append(token)
    return ", ".join(mc for mc in mentioned_companies)

In [1188]:
# . Earnings didn’t go how we wanted but used time and finally got out of LOGI and with a nice profit 🤑'
#f_com = 'This is from a <@579205396166737921> earning play way back when.'
f_com = "Just a little amzn play lol"

In [1189]:
sent = sent_tokenize(f_com)
for s in sent:
    tokens = word_tokenize(s)
    print(pos_tag(tokens))
    for token in tokens:
        token = token.lower()
        if (token not in stop_words) and (token in ticker_translater.keys()):
            print(token)

[('Just', 'RB'), ('a', 'DT'), ('little', 'JJ'), ('amzn', 'JJ'), ('play', 'NN'), ('lol', 'NN')]
amzn
play


In [1190]:
f_df = final_dataframe.assign(mentionedStocks = lambda x: x.content.apply(findCompanies))

In [1191]:
caught_mentions = f_df.mentionedStocks.values

In [1192]:
mentions = {}
for cm in caught_mentions:
    if "," in cm:
        words = cm.split(",")
        for word in words:
            word = word.strip()
            if word not in mentions.keys():
                mentions[word] = 1
            else:
                mentions[word] += 1
    elif cm != "":
        cm = cm.strip()
        if cm not in mentions.keys():
            mentions[cm] = 1
        else:
            mentions[cm] += 1
    else:
        next

In [1193]:
mentions

{'beke': 1,
 'acb': 5,
 'mid': 2,
 'pltr': 2,
 'amd': 56,
 'tsla': 82,
 'spy': 139,
 'nio': 15,
 'ba': 26,
 'sp': 3,
 'clsk': 1,
 'nflx': 13,
 'pton': 15,
 'ive': 27,
 'g': 2,
 'feye': 4,
 'pins': 5,
 'car': 2,
 'cold': 1,
 'zm': 13,
 'fund': 2,
 'gold': 4,
 'dkng': 2,
 'bynd': 51,
 'cost': 10,
 'plug': 6,
 'pdd': 1,
 'amzn': 20,
 'sino': 1,
 'baba': 17,
 'lyft': 1,
 'uve': 1,
 'b': 2,
 'qcom': 4,
 'ig': 2,
 'snow': 1,
 'intc': 5,
 'acc': 2,
 'vc': 1,
 'jblu': 1,
 'snap': 10,
 'road': 2,
 'tmo': 1,
 'logi': 6,
 'gem': 1,
 'asln': 1,
 'sgbx': 1,
 'nvda': 27,
 'aapl': 30,
 'thr': 1,
 'roku': 36,
 'hyln': 1,
 'fsly': 3,
 'ups': 3,
 'eem': 2,
 'se': 1,
 'nke': 1,
 'iwn': 1,
 'bkng': 6,
 'calm': 3,
 'jpm': 4,
 'ibb': 1,
 'regn': 6,
 'ddog': 1,
 'wkhs': 2,
 'care': 3,
 'bbby': 2,
 'wynn': 8,
 'fb': 19,
 'exas': 30,
 'trip': 2,
 'lulu': 7,
 'mu': 7,
 'fdx': 6,
 'eod': 3,
 'wrap': 2,
 'nee': 1,
 'main': 2,
 'xlf': 2,
 'msft': 12,
 'ulta': 4,
 'crm': 7,
 'nkla': 2,
 'gm': 3,
 'hes': 2,
 'jamf':

In [1194]:
ment_sort = dict(sorted(mentions.items(), key=lambda item: item[1]))

In [1195]:
ment_sort

{'beke': 1,
 'clsk': 1,
 'cold': 1,
 'pdd': 1,
 'sino': 1,
 'lyft': 1,
 'uve': 1,
 'snow': 1,
 'vc': 1,
 'jblu': 1,
 'tmo': 1,
 'gem': 1,
 'asln': 1,
 'sgbx': 1,
 'thr': 1,
 'hyln': 1,
 'se': 1,
 'nke': 1,
 'iwn': 1,
 'ibb': 1,
 'ddog': 1,
 'nee': 1,
 'jamf': 1,
 'uk': 1,
 'pace': 1,
 'sum': 1,
 'five': 1,
 'vz': 1,
 'penn': 1,
 'mt': 1,
 'wifi': 1,
 'tlry': 1,
 'jmia': 1,
 'glw': 1,
 'imo': 1,
 'gs': 1,
 'ncno': 1,
 'ev': 1,
 'goog': 1,
 'ads': 1,
 'jp': 1,
 'ino': 1,
 'jwn': 1,
 'uonek': 1,
 'docu': 1,
 'ivr': 1,
 'mgm': 1,
 'mom': 1,
 'rtx': 1,
 'meli': 1,
 'salt': 1,
 'rcl': 1,
 'upro': 1,
 'opp': 1,
 'stor': 1,
 'bj': 1,
 'ni': 1,
 'xlk': 1,
 'prty': 1,
 'dri': 1,
 'wmt': 1,
 'gld': 1,
 'kmb': 1,
 'lvs': 1,
 'solo': 1,
 'xspa': 1,
 'lit': 1,
 'sfix': 1,
 'googl': 1,
 'form': 1,
 'must': 1,
 'csco': 1,
 'hyg': 1,
 'nail': 1,
 'mrk': 1,
 'vice': 1,
 'fan': 1,
 'plnt': 1,
 'dow': 1,
 'bw': 1,
 'siri': 1,
 'spi': 1,
 'ofc': 1,
 'sub': 1,
 'de': 1,
 'six': 1,
 'mos': 1,
 'aig': 1,
 'he

In [1196]:
len(ment_sort)

280

In [1185]:
for c in comments:
    if " k " in c.lower():
        print(c)

quick k on BA
1k to 6 k in two  weeks! Trading futures.
1.5 k on pypl 💵💵💵
Plus over 2 k off roku that I could not find to post
I know that k you analyst


In [1197]:
pos_tag(word_tokenize('I saw that was going to cost me alot but got side track, would of been great for me haha <@!579205396166737921>'))

[('I', 'PRP'),
 ('saw', 'VBD'),
 ('that', 'DT'),
 ('was', 'VBD'),
 ('going', 'VBG'),
 ('to', 'TO'),
 ('cost', 'VB'),
 ('me', 'PRP'),
 ('alot', 'RB'),
 ('but', 'CC'),
 ('got', 'VBD'),
 ('side', 'JJ'),
 ('track', 'NN'),
 (',', ','),
 ('would', 'MD'),
 ('of', 'IN'),
 ('been', 'VBN'),
 ('great', 'JJ'),
 ('for', 'IN'),
 ('me', 'PRP'),
 ('haha', 'JJ'),
 ('<', 'JJ'),
 ('@', 'NN'),
 ('!', '.'),
 ('579205396166737921', 'CD'),
 ('>', 'NN')]

In [1107]:
{"ath ": "all time High",
 "pdt": "pattern day trader",
"rh": "RobinHood",
"td": "TD Ameritrade", 
"tos": "Think or Swim",
"leg": "a trade",
"pump": "overvalue",
"exp": "expiration",
"dd": "due diligence"}

{'ath ': 'all time High',
 'pdt': 'pattern day trader',
 'rh': 'RobinHood',
 'td': 'TD Ameritrade',
 'tos': 'Think or Swim',
 'leg': 'a trade',
 'pump': 'overvalue',
 'exp': 'expiration',
 'dd': 'due diligence'}

In [669]:
pos_tag(word_tokenize('little play'))

[('little', 'JJ'), ('play', 'NN')]

In [674]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [1169]:
joer.translate(str.maketrans('', '', string.punctuation))

'looney bernie sanders Cannot take me'

In [1198]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 2 commits.
  (use "git push" to publish your local commits)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	CommentProcessFlow.ipynb
	data/listedCompanies/

nothing added to commit but untracked files present (use "git add" to track)
